<a href="https://colab.research.google.com/github/Om-Tiwari/GraphRAG-chatDoctor/blob/main/GRAPH_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade --quiet langchain jq langchain-community langchain-huggingface langchain-groq langchain-experimental neo4j tiktoken yfiles_jupyter_graphs sentence_transformers
!pip -q install nltk rouge-score sentence-transformers scikit-learn python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 440.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.1/208.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [6]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from pydantic import BaseModel, Field
from typing import Tuple, List, Optional
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain_community.document_loaders import JSONLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_groq import ChatGroq
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.runnables import ConfigurableField, RunnableParallel, RunnablePassthrough

# Importing necessary for testing libraries
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import Levenshtein

try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

from tqdm.autonotebook import tqdm, trange

In [ ]:
os.environ["NEO4J_URI"] = "neo4j+s://xxx.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "<password>"
os.environ["GROQ_API_KEY"] = "gsk_..."

In [10]:
loader = JSONLoader(
    file_path="/content/train_2kchat.jsonl",
    jq_schema=".chat",
    text_content=False,
    json_lines=True,
)

docs = loader.load()
# docs_length = 2000 JSON

# Define chunking strategy
text_splitter = TokenTextSplitter(chunk_size=1024, chunk_overlap=24)
documents = text_splitter.split_documents(docs)
print(docs[51])
len(documents)


page_content='[{'input': [{'role': 'user', 'content': 'Hello doctor, I have been going to a dentist about a tooth that is sore. They started a root canal but then did not finish. I have gone back several times and every time they open up the tooth and insert medicine. My tooth is in agony right now. I am just wondering if this is a standard procedure during a root canal? Should not the dentist remove the nerves so I am not in so much pain? Or is it the standard procedure to fight the infection first and then remove the nerves?'}], 'ideal': 'Hello. I understand your concern. The number of sittings in the root canal procedure depends on the amount and extent of infection. If the infection is more, the tooth is opened up and medicated  ChatDoctor.  Antibiotics and painkillers will be prescribed for the next three days to control the pain and infection. After three days, the root canal treatment can be done painlessly, once the nerve had died. From your query and the details provided, it s

2000

In [ ]:
llm = ChatGroq(temperature=0, model_name="llama-3.1-70b-versatile")

prompt = ChatPromptTemplate.from_template((
  """
You are a knowledge graph engineer working on an AI Doctor project that aims to extract structured medical data from patient inquiries and responses. Your task is to extract key information relevant to the patient's medical condition from the following text input.

Instructions for Extraction:

Disease: Identify the primary medical condition or diagnosis mentioned in the user query.
Symptoms: List the symptoms experienced by the patient, specifying any severity or duration provided.
Treatment History: Document any treatments or medications the patient has received, including dosages and duration.
Medical History: Include any relevant past medical history that may impact the current condition.
Family History: Note any pertinent family medical history if mentioned.
Lifestyle Factors: Identify any lifestyle information, such as substance use (e.g., drugs, alcohol), diet, or exercise habits.
Recommendations: Include any recommendations or advice given by the doctor in the ideal response.
  """
  )
)


llm_transformer = LLMGraphTransformer(
    llm=llm,
    )

In [ ]:
graph = Neo4jGraph()

graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [ ]:
graph.add_graph_documents(
    graph_documents=[],
    baseEntityLabel=True,
    include_source=True
)

In [ ]:
# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t"

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    #display(widget)
    return widget

showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

In [ ]:
vector_index = Neo4jVector.from_existing_graph(
    HuggingFaceEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Retriever

graph.query(
    "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the Cases, Drug, Reaction, Outcome, Therapy, AgeGroup, Symptom, Condition, Diagnosis, Treatment appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a friendly senior medical expert. Please answer the medical questions based on the patient's description in a Natural way, include your resoning on how you came to the conclusion to problem conclusion you have come into, your response format will be in two section
            `response`
            \n
            `reasoning`
            """,
        ),
        (
            "human",
            "input: {question}",
        ),
    ]
)


entity_chain = prompt | llm.with_structured_output(Entities)

In [ ]:
q = """
This morning, I woke up feeling like the room was violently spinning every time I tried to move, especially when I sat up or changed positions.
It wasn’t a mild dizziness—it was intense enough that I had to hold on to things to avoid falling.
When I tried to walk to the bathroom, I felt incredibly unsteady, almost like I couldn’t control my balance, and this triggered a wave of nausea.
I thought I might vomit, but I couldn’t. I also noticed a slight pressure in my ear, though it’s not painful.
I took some painkillers and slept for a few hours, but the sensation didn’t improve much. Oddly enough, if I stay completely still—lying down or sitting—it seems fine.
But as soon as I try to move, especially getting up or turning my head, the dizziness is unbearable. I’ve had some mild stomach discomfort and bloating too, though I’m not sure if that’s related.
Strangely, after I went to the bathroom, the dizziness seemed to decrease slightly, but it quickly returned when I moved again. Could this be connected to my ear or something else entirely? I’m concerned because it’s persistent.
"""

In [ ]:
entity_chain.invoke({"question": q}).names

['dizziness',
 'nausea',
 'vomiting',
 'ear pressure',
 'stomach discomfort',
 'bloating',
 'balance control',
 'spinning sensation',
 'unsteadiness',
 'painkillers',
 'persistent dizziness']

In [ ]:
def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similarity threshold (~2 changed characters) to each word, then combines
    them using the AND operator. Useful for mapping entities from user questions
    to database values, and allows for some misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

# Fulltext index query
def structured_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [ ]:
print(structured_retriever(q))

In [ ]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [ ]:
# Condense a chat history and follow-up question into a standalone question
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""  # noqa: E501
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatMistralAI(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
question = input()
while question != "exit":
    res = chain.invoke({"question": question, "chat_history": chat_history})
    print(res)
    chat_history.append((question, str(res)))
    question = input()

In [ ]:
res = chain.invoke({"question": q})
print(res)

Search query: This morning, I woke up feeling like the room was violently spinning every time I tried to move, especially when I sat up or changed positions. It wasn’t a mild dizziness—it was intense enough that I had to hold on to things to avoid falling. When I tried to walk to the bathroom, I felt incredibly unsteady, almost like I couldn’t control my balance, and this triggered a wave of nausea. I thought I might vomit, but I couldn’t. I also noticed a slight pressure in my ear, though it’s not painful. I took some painkillers and slept for a few hours, but the sensation didn’t improve much. Oddly enough, if I stay completely still—lying down or sitting—it seems fine. But as soon as I try to move, especially getting up or turning my head, the dizziness is unbearable. I’ve had some mild stomach discomfort and bloating too, though I’m not sure if that’s related. Strangely, after I went to the bathroom, the dizziness seemed to decrease slightly, but it quickly returned when I moved ag

Hello. I read carefully your question and would recommend consulting first with an ENT specialist in order to exclude a possible inner ear disorder, which could trigger this clinical situation. The symptoms you described, such as intense dizziness, nausea, and pressure in the ear, are suggestive of a possible inner ear issue.


In [ ]:
# Define the function to calculate all metrics
def evaluate_all_metrics(reference, hypothesis):
    results = {}

    # 1. BLEU Score
    reference_tokens = [reference.split()]  # BLEU expects a list of reference sentences
    hypothesis_tokens = hypothesis.split()
    bleu_score = sentence_bleu(reference_tokens, hypothesis_tokens)
    results['BLEU'] = bleu_score

    # 2. ROUGE Scores
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge_scores = scorer.score(reference, hypothesis)
    results['ROUGE-1'] = rouge_scores['rouge1'].fmeasure  # F1 score for unigram overlap
    results['ROUGE-L'] = rouge_scores['rougeL'].fmeasure  # F1 score for longest common subsequence

    # 3. Cosine Similarity (Semantic)
    model = SentenceTransformer('sentence-transformers/msmarco-distilbert-cos-v5')  # Small and efficient model
    embeddings = model.encode([reference, hypothesis])
    cosine_sim = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
    results['Cosine Similarity'] = cosine_sim

    # 4. Levenshtein Distance
    levenshtein_distance = Levenshtein.distance(reference, hypothesis)
    results['Levenshtein Distance'] = levenshtein_distance

    return results

In [ ]:
ideal_res_doc, question_user, response = [], [], []
print(len(docs))
for i in range(len(docs)):
  text = docs[i].page_content
  if 'content\':' in text:
    extracted_text = text.split('content\':')[1].strip()
    split_parts = extracted_text.split("'ideal':")
    query = split_parts[0].strip()[:-3].strip()
    ideal_response = split_parts[1].strip()[:-2].strip()
    question_user.append(query)
    ideal_res_doc.append(ideal_response)

2000
SUiiii -----> 0
SUiiii -----> 1
SUiiii -----> 2
SUiiii -----> 3
SUiiii -----> 4
SUiiii -----> 5
SUiiii -----> 6
SUiiii -----> 7
SUiiii -----> 8
SUiiii -----> 9
SUiiii -----> 10
SUiiii -----> 11
SUiiii -----> 12
SUiiii -----> 13
SUiiii -----> 14
SUiiii -----> 15
SUiiii -----> 16
SUiiii -----> 17
SUiiii -----> 18
SUiiii -----> 19
SUiiii -----> 20
SUiiii -----> 21
SUiiii -----> 22
SUiiii -----> 23
SUiiii -----> 24
SUiiii -----> 25
SUiiii -----> 26
SUiiii -----> 27
SUiiii -----> 28
SUiiii -----> 29
SUiiii -----> 30
SUiiii -----> 31
SUiiii -----> 32
SUiiii -----> 33
SUiiii -----> 34
SUiiii -----> 35
SUiiii -----> 36
SUiiii -----> 37
SUiiii -----> 38
SUiiii -----> 39
SUiiii -----> 40
SUiiii -----> 41
SUiiii -----> 42
SUiiii -----> 43
SUiiii -----> 44
SUiiii -----> 45
SUiiii -----> 46
SUiiii -----> 47
SUiiii -----> 48
SUiiii -----> 49
SUiiii -----> 50
SUiiii -----> 51
SUiiii -----> 52
SUiiii -----> 53
SUiiii -----> 54
SUiiii -----> 55
SUiiii -----> 56
SUiiii -----> 57
SUiiii -----> 58
SU

In [ ]:
cosSum, bleuSum, rougeSum, LevenshteinSum = 0, 0, 0, 0
test_size = 100

for i in range(test_size):
  res = chain.invoke({"question": question_user[i]})
  response.append(res)
  metrics = evaluate_all_metrics(ideal_res_doc[i], response[i])
  cosSum += metrics['Cosine Similarity']
  bleuSum += metrics['BLEU']
  rougeSum += metrics['ROUGE-L']
  LevenshteinSum += metrics['Levenshtein Distance']

  print(f"Done-> {i}")


Search query: 'Hello doctor,I had mumps five months ago and after that, I started to have an infection in my left testes. It was swollen and now it has shrunk to almost half the size of the other one. As I am sexually active, I feel a pain in each of the vas deferens after sex. If I do not have sex for days, they become sensitive. I was treated with Ceftum 500 mg, the first time I had an infection. Now my question is, is there any chance that the infection is still in my body? And, do I need to get examined for it? For the time being, please suggest some precautionary antibiotics for my relief.'


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Done-> 0
Search query: 'Hello doctor,I am trying to conceive but my husband and I did cocaine a week ago. How long should my husband and I wait to safely continue to try to get pregnant? How long until it is out of our system? How long does cocaine stay in sperm? Thanks in advance.'


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Done-> 1
Search query: 'Hello doctor,I am currently on the Yaz contraceptive pill. I skipped my sugar pills for around four months and had a breakthrough period this month. I had my period last week, but I still have some residual brown discharge for about a week now. Additionally, I had sex a few days ago and bled during it. Nothing hurts or is sore, I just want to know how long I am going to have the brown discharge and if I should be worried or if I should do anything about it?'


...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Done-> 11
Search query: 'Hello doctor,I am a 50-year-old female who has had an elevated ferritin level for many years now. My last ferritin level was 342, 20 days back. My ferritin level has been as high as 390. My TIBC, UIBC, iron, and iron saturation have always been within normal range. My glucose has always ranged from 100 to 104. I was tested for hemochromatosis and results are negative. My doctors keep telling me that we will just watch my ferritin level and I have bloodwork done every six months. I was diagnosed with fibromyalgia last year. I am looking for some answers and some help.'


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Done-> 12
Search query: 'Hello doctor,I have had a fishy odor down there since I was in elementary school. I was not sexually active at all either. Just a couple years ago I realized I might have something called bacterial vaginosis. I went to the doctor for it a couple years ago and got prescribed Metronidazole and it did nothing for me. Ever since then I have not really gone to the doctor about it again. I am worried because I have read that BV left untreated can make you infertile. How common is it that this could happen? And do you have any tips for me?'


KeyboardInterrupt: 

In [ ]:
len(response)
response[-1]

'Hello. Elevated ferritin could be from congenital hemochromatosis but HFE gene mutation is negative, so it is excluded. Secondly, a hemolytic condition also can be a possibility but serum LDH (lactic acid dehydrogenase) is needed to confirm. If you are having repeated blood transfusion history then provide history regarding that. I suggest you investigate with transferrin saturation as well next time when you investigate for ferritin. Do not worry about high serum ferritin as all other related parameters are also within range. If ferritin constantly remains elevated than phlebotomy can be done to decrease the ferritin level. Once transferrin saturation investigation has been done, provide history regarding that. Take care. For more information consult a hematologist online.'

In [ ]:
print("Avg Cosine Similarity:", cosSum/test_size)
print("Avg BLEU Score:", bleuSum/test_size)
print("Avg ROUGE Score:", rougeSum/test_size)
print("Avg Levenshtein Distance:", LevenshteinSum/test_size)

Avg Cosine Similarity: 0.9594249083445623
Avg BLEU Score: 0.809880547025691
Avg ROUGE Score: 0.9274377926516716
Avg Levenshtein Distance: 82.61538461538461


In [ ]:
ideal_res_doc[12]

"'Hello. Elevated ferritin could be from congenital hemochromatosis but HFE gene mutation is negative, so it is excluded (attachment removed to protect patient identity). Secondly, a hemolytic condition also can be a possibility but serum LDH (lactic acid dehy ChatDoctor.  If you are having repeated blood transfusion history then provide history regarding that.  I suggest you investigate with transferrin saturation as well next time when you investigate for ferritin. Do not worry about high serum ferritin as all other related parameters are also within range. If ferritin constantly remains elevated than phlebotomy can be done to decrease the ferritin level. Once transferrin saturation investigation has been done, provide history regarding that. Take care. For more information consult a hematologist online '"

In [ ]:
response[]

SyntaxError: invalid syntax (<ipython-input-145-90e00f8cba21>, line 1)

In [ ]:
res = chain.invoke({"question": 'Hello I am 18 yo girl. I have five brown marks, two of them are 2 cm size and the other three of 0.5 cm. They are irregular in shape. Initial two marks appeared when I was 3 months old and now when I am 18 year old three more for added that are comparatively light and 0.5 cm of size. Are these birthmarks? Will they fade with the time? Do we need to take any precautions so that they do not increase or get added further? Can we also provide any medication to cure them?'})
# response.append(res)
metrics = evaluate_all_metrics(ideal_res_doc[49], res)
print("\n\n")
print(metrics['Cosine Similarity'])
print(metrics['BLEU'])
print(metrics['ROUGE-L'])
print(metrics['Levenshtein Distance'])
print(question_user[49])
print(res)
print(ideal_res_doc[49])


Search query: Hello I am 18 yo girl. I have five brown marks, two of them are 2 cm size and the other three of 0.5 cm. They are irregular in shape. Initial two marks appeared when I was 3 months old and now when I am 18 year old three more for added that are comparatively light and 0.5 cm of size. Are these birthmarks? Will they fade with the time? Do we need to take any precautions so that they do not increase or get added further? Can we also provide any medication to cure them?


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(





0.82958823
0.4397645271696075
0.6947368421052632
98
'Hello doctor,We have a 9-month-old baby girl. She has five brown marks, two of them are 2 cm size and the other three of 0.5 cm. They are irregular in shape. Initial two marks appeared when she was 3 months old and now when she is 9 months three more for added that are comparatively light and 0.5 cm of size. Are these birthmarks? Will they fade with the time? Do we need to take any precautions so that they do not increase or get added further? Can we also provide any medication to cure them?'
Hi. On the basis of history, it seems to be cafe lait macules. However, I need photographs of the lesion to be sure. Presently, I will not suggest any medications as these are generally harmless. Kindly review with photographs. Take care.
'Hi. Please send me a photograph of the lesion. On the basis of history, it seems to be cafe lait macules, but to be sure I need photographs of the lesion. Presently, I will not suggest any medications as ba